# LGBM - Training

Sources
1. [LGBM & Deberta Explained by ZULQARNAIN ALI](https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained)

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
import pickle as pkl
import wandb
from lightgbm import log_evaluation, early_stopping
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.paths import Paths
from lib.utils.utils import seed_everything
from lib.criterion.metrics import log_metrics
from lib.model.utils import quadratic_weighted_kappa, qwk_obj, get_score

In [4]:
seed_everything()

### Setting Up WandB

In [5]:
WANDB_PROJECT = "Kaggle_ASE_2.0"
WANDB_NAME = f"LGBM-ASE-11"

In [6]:
wandb.login(key=os.environ.get('WANDB_API_KEY'))
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_NAME,
    save_code=True,
    job_type="train",
    config=config,
)
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shakleenishfar (laplacesdemon43). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ishfar/.netrc


## Model Training

### Dataset Creation

In [7]:
df = pd.read_csv(Paths.FEATURE_ENGG_CSV_PATH)
df.shape

(17307, 19381)

In [8]:
ids = df.loc[:, "essay_id"]
X, y = df.drop(columns=["score", "essay_id"]), df.loc[:, "score"]
del df

X.shape, y.shape

((17307, 19379), (17307,))

### Callbacks

In [9]:
callbacks = [
    log_evaluation(period=25),
    early_stopping(stopping_rounds=75, first_metric_only=True),
]

### Constants

In [10]:
a = config.lgbm_a
b = config.lgbm_b

### Training Loop

In [11]:
def train_loop(X_train, y_train, X_valid, y_valid):
    y_train = y_train.astype(np.float32) - a
    y_valid = y_valid.astype(np.float32) - a

    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics="None",
        learning_rate=0.01,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.3,
        reg_alpha=0.7,
        reg_lambda=0.1,
        n_estimators=700,
        random_state=config.random_seed,
        extra_trees=True,
        class_weight="balanced",
        n_jobs=6,
        verbosity=0,
    )

    # Train model
    predictor = model.fit(
        X_train,
        y_train,
        eval_names=["train", "valid"],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks,
    )

    return model, predictor

In [12]:
def validate_model(idx, X_valid, y_valid, id_valid, predictor):
    y_valid_int = y_valid.astype(np.int64).to_numpy()
    preds = predictor.predict(X_valid) + a
    preds = preds.clip(1, 6).round().astype(np.int64)

    oof = pd.DataFrame(
        {
            "essay_id": id_valid,
            "score": y_valid_int - 1,
            "pred_score": preds - 1,
        }
    )
    log_metrics(oof, f"Fold {idx}")

    return oof

### Fold Loop

In [13]:
skf = StratifiedKFold(
    n_splits=config.lgbm_n_folds,
    random_state=config.random_seed,
    shuffle=True,
)

In [14]:
oof_df = pd.DataFrame()

for idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
    id_valid = ids.loc[valid_idx].to_numpy()

    model, predictor = train_loop(X_train, y_train, X_valid, y_valid)

    with open(f"output/LGBM/{idx}.pkl", "wb") as file:
        pkl.dump({"predictor": predictor, "model": model}, file)

    oof = validate_model(idx, X_valid, y_valid, id_valid, predictor)
    oof_df = pd.concat([oof_df, oof])

[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.445845	valid's QWK: 0.478024
[50]	train's QWK: 0.735721	valid's QWK: 0.751619
[75]	train's QWK: 0.798084	valid's QWK: 0.806355
[100]	train's QWK: 0.834671	valid's QWK: 0.841421
[125]	train's QWK: 0.858527	valid's QWK: 0.86505
[150]	train's QWK: 0.868501	valid's QWK: 0.875763
[175]	train's QWK: 0.872081	valid's QWK: 0.880666
[200]	train's QWK: 0.875253	valid's QWK: 0.885677
[225]	train's QWK: 0.877606	valid's QWK: 0.890765
[250]	train's QWK: 0.880345	valid's QWK: 0.892712
[275]	train's QWK: 0.881992	valid's QWK: 0.893371
[300]	train's QWK: 0.882984	valid's QWK: 0.895274
[325]	train's QWK: 0.884392	valid's QWK: 0.894643
[350]	train's QWK: 0.885169	valid's QWK: 0.895337
[375]	train's QWK: 0.885814	valid's QWK: 0.895184
[400]	train's QWK: 0.886439	valid's QWK: 0.894587
Early stopping, best iteration is:
[341]	train's QWK: 0.885041	valid's QWK: 0.89638
Eva

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.456039	valid's QWK: 0.446883
[50]	train's QWK: 0.728751	valid's QWK: 0.695712
[75]	train's QWK: 0.796976	valid's QWK: 0.77586
[100]	train's QWK: 0.839037	valid's QWK: 0.82289
[125]	train's QWK: 0.862377	valid's QWK: 0.845231
[150]	train's QWK: 0.870352	valid's QWK: 0.856918
[175]	train's QWK: 0.874548	valid's QWK: 0.858153
[200]	train's QWK: 0.877216	valid's QWK: 0.860348
[225]	train's QWK: 0.87972	valid's QWK: 0.864754
[250]	train's QWK: 0.881633	valid's QWK: 0.863117
[275]	train's QWK: 0.883512	valid's QWK: 0.865907
[300]	train's QWK: 0.884163	valid's QWK: 0.865498
[325]	train's QWK: 0.885431	valid's QWK: 0.865659
Early stopping, best iteration is:
[271]	train's QWK: 0.883165	valid's QWK: 0.866299
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK:

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.43502	valid's QWK: 0.43494
[50]	train's QWK: 0.739398	valid's QWK: 0.737532
[75]	train's QWK: 0.798891	valid's QWK: 0.797687
[100]	train's QWK: 0.83971	valid's QWK: 0.835606
[125]	train's QWK: 0.860168	valid's QWK: 0.863044
[150]	train's QWK: 0.868793	valid's QWK: 0.868899
[175]	train's QWK: 0.873087	valid's QWK: 0.87017
[200]	train's QWK: 0.875627	valid's QWK: 0.872326
[225]	train's QWK: 0.878718	valid's QWK: 0.876927
[250]	train's QWK: 0.880871	valid's QWK: 0.881055
[275]	train's QWK: 0.882714	valid's QWK: 0.882924
[300]	train's QWK: 0.883693	valid's QWK: 0.885873
[325]	train's QWK: 0.885175	valid's QWK: 0.887234
[350]	train's QWK: 0.885808	valid's QWK: 0.88784
[375]	train's QWK: 0.886463	valid's QWK: 0.887554
[400]	train's QWK: 0.886847	valid's QWK: 0.888604
[425]	train's QWK: 0.887624	valid's QWK: 0.888604
[450]	train's QWK: 0.888273	valid's QWK: 

In [15]:
score = get_score(oof_df["score"], oof_df["pred_score"])
print(f'Overall Score: {score:<.4f}')
wandb.log({"CV/qwk_score": score})

Overall Score: 0.8852


In [16]:
wandb.log({"oof_table": wandb.Table(dataframe=oof_df)})

In [17]:
log_metrics(
    pd.DataFrame(
        {
            "score": oof_df.score.to_numpy(),
            "pred_score": oof_df.pred_score.to_numpy(),
        }
    ),
    "Overall",
)

In [18]:
wandb.finish()

CV/qwk_score,▁
CV/qwk_score,0.88525
